In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import os

files = []

for i in os.listdir():
    if i.endswith('.csv'):
        files.append(i)
        
print(files)

['서울특별시 (안심이) CCTV 설치 현황.csv', 'allllll.csv', 'Full_PCA.csv']


In [3]:
data = pd.read_csv(files[0], encoding = 'cp949')

In [4]:
data

,자치구,안심 주소,CCTV 용도,위도,경도,CCTV 수량,수정 일시
0,중랑구,방범373 충신길 53 메인1,도심공원,37.575179,127.007360,1,2021-11-16
1,중랑구,방범459 백석동길 187 검지2,도심공원,37.596453,126.967939,1,2021-11-16
2,중랑구,방범128 돈화문로11다길 44 검지1,도심공원,37.573735,126.989771,1,2021-11-16
3,중랑구,방범398 종로51가길 31 메인1,도심공원,37.573467,127.012433,1,2021-11-16
4,중랑구,방범458 삼청로5길 30 검지3,도심공원,37.582970,126.980254,1,2021-11-16
...,...,...,...,...,...,...,...
59419,강남구,[역삼1-401] (2) 역삼1동796-30_3,방범,37.493211,127.039474,1,2022-03-14
59420,강남구,[논현2-218] (6) 논현로120길6(221-1)_2,방범,37.510870,127.033140,1,2022-03-14
59421,강남구,[논현2-218] (8) 논현로120길6(221-1)_1,방범,37.510870,127.033140,1,2022-03-14
59422,강남구,[역삼1-401] (2) 역삼1동796-30_4,방범,37.493211,127.039474,1,2022-03-14


In [5]:
lat = data['위도']
long = data['경도']

In [6]:
location = pd.concat([lat, long], axis = 1)

In [7]:
location

,위도,경도
0,37.575179,127.007360
1,37.596453,126.967939
2,37.573735,126.989771
3,37.573467,127.012433
4,37.582970,126.980254
...,...,...
59419,37.493211,127.039474
59420,37.510870,127.033140
59421,37.510870,127.033140
59422,37.493211,127.039474


In [8]:
import json

In [9]:
headers = {"Authorization": "KakaoAK 720504ed1f0f3c9f3e3ef467230648d7"}

In [10]:
location['address_name'] = ''
location['구'] = ''
location['행정동'] = ''

In [11]:
counter = 0

for i in tqdm(location.itertuples()):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(i[2], i[1])
    counter += 1
    try:
        one_response = requests.request('GET', url, headers = headers)
        one_json = one_response.json()
        b = one_json.get('documents')
        a = b[1]
        row = i.Index
        location.at[row, 'address_name'] = a['address_name']
        location.at[row, '구'] = a['region_2depth_name']
        location.at[row, '행정동'] = a['region_3depth_name']
    except:
        print('Error at {}'.format(counter - 1))
        break

59424it [1:27:52, 11.27it/s]


In [12]:
#location.to_csv('CCTV_location.csv', encoding = 'cp949')